---
# Algoritmo de McEliece
---

---
#### Definicion el cuerpo y polinomio de Goppa

In [1]:
rango = 3;
show(rango)

3

In [2]:
# Definimos el cuerpo donde vamos a trabajar
m = 4;
N = 2^m
K_.<a> = GF(2)
F.<a> = GF(2^m)

In [3]:
# Creamos el anillo de polinomios
PR = PolynomialRing(F,'X')
X = PR.gen()
g = X^3+X+1
L = [a^i for i in range(N)]

In [4]:
show(K_)
show(F)
show(PR)
show(g)
show(L)

Finite Field of size 2

Finite Field in a of size 2^4

Univariate Polynomial Ring in X over Finite Field in a of size 2^4

X^3 + X + 1

[1,
 a,
 a^2,
 a^3,
 a + 1,
 a^2 + a,
 a^3 + a^2,
 a^3 + a + 1,
 a^2 + 1,
 a^3 + a,
 a^2 + a + 1,
 a^3 + a^2 + a,
 a^3 + a^2 + a + 1,
 a^3 + a^2 + 1,
 a^3 + 1,
 1]

---
## Funcion descomponer polinomio

In [5]:
# -------------------
# Dado un polinomio, permite descomponerlo en factores irreducibles
# -------------------
def descomponer_polinomio(p):

    Phi1 = p.parent()

    p0 = Phi1([sqrt(c) for c in p.list()[0::2]])
    p1 = Phi1([sqrt(c) for c in p.list()[1::2]])
    return (p0,p1)



#### Traza funcion descomponer polinomios


In [6]:
def descomponer_polinomio_traza(p):

    Phi1 = p.parent()
    show(Phi1)
    print("Phi1: ")
    print(Phi1)
    print(p.list())

    print("primer polinomio: ")
    print(p.list()[0::2])
    p0 = Phi1([sqrt(c) for c in p.list()[0::2]])
    Phi1([print(sqrt(c)) for c in p.list()[0::2]])

    print("Segundo polinomio: ")
    print(p.list()[1::2])
    p1 = Phi1([sqrt(c) for c in p.list()[1::2]])
    Phi1([print(sqrt(c)) for c in p.list()[1::2]])

    return (p0,p1)

In [7]:
print("Polinomio: ")
show(g)
print("Traza descomponer: ")
show(descomponer_polinomio_traza(g))

Polinomio: 


X^3 + X + 1

Traza descomponer: 


Univariate Polynomial Ring in X over Finite Field in a of size 2^4

Phi1: 
Univariate Polynomial Ring in X over Finite Field in a of size 2^4
[1, 1, 0, 1]
primer polinomio: 
[1, 0]
1
0
Segundo polinomio: 
[1, 1]
1
1


(1, X + 1)

---
## Euclides extendido

In [8]:
def algoritmo_euclides_extendido(self, other):
    delta = self.degree()

    if other.is_zero():
        ring = self.parent()
        return self, R.one(), R.zero()

    ring = self.parent()
    a = self
    b = other

    s = ring.one()
    t = ring.zero()

    resto0 = a
    resto1 = b

    while true:
        cociente,resto_auxiliar = resto0.quo_rem(resto1)
        resto0 = resto1
        resto1 = resto_auxiliar

        s = t
        t = s - t*cociente

        if resto1.degree() <= floor((delta-1)/2) and resto0.degree() <= floor((delta)/2):
             break

    V = (resto0-a*s)//b
    coeficiente_lider = resto0.leading_coefficient()

    return resto0/coeficiente_lider, s/coeficiente_lider, V/coeficiente_lider


---
## Inversa de un polinomio: algoritmo de Euclides de Sage

In [9]:
# -------------------
# Calcula la inversa de un polinomio con ayuda del algoritmo declarado en Sage
# -------------------
def inversa_g(p,g):
    (d,u,v) = xgcd(p,g)
    return u.mod(g)

--- 
## Algoritmo de Patterson

In [10]:
def decodePatterson(y):

    alpha = vector(H*y)

    polinomioS = PR(0)
    for i in range(len(alpha)):
        polinomioS = polinomioS + alpha[i]*(X^(len(alpha)-i-1))

    vector_g = descomponer_polinomio(g)
    w = ((vector_g[0])*inversa_g(vector_g[1],g)).mod(g)
    vector_t = descomponer_polinomio(inversa_g(polinomioS,g) + X)

    R = (vector_t[0]+(w)*(vector_t[1])).mod(g)

    (a11,b11,c11) = algoritmo_euclides_extendido(g,R)


    sigma = a11^2+X*(c11^2)

    for i in range(N):
        if (sigma(a^i)==0):
            print ("Error encontrado en la posición: " + str(i))
            y[i] = y[i] + 1
    return y

---
## Matriz T

In [11]:
T = matrix(F,rango,rango)
for i in range(rango):
    count = rango - i
    for j in range(rango):
        if i > j:
            T[i,j]=g.list()[count]
            count = count + 1
        if i < j:
            T[i,j] = 0
        if i == j:
            T[i,j] = 1

In [12]:
print ("Matriz T: ")
show(T)
show(F)
show(g.list())
show(g)

Matriz T: 


[1 0 0]
[0 1 0]
[1 0 1]

Finite Field in a of size 2^4

[1, 1, 0, 1]

X^3 + X + 1

---
## Matriz V

In [13]:
V = matrix([[L[j]^i for j in range(N)] for i in range(rango)])
print ("Matriz V: ")
show(V)
print("Conjunto del Cuerpo de Goppa: ")
show(L)

Matriz V: 


[                1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1                 1]
[                1                 a               a^2               a^3             a + 1           a^2 + a         a^3 + a^2       a^3 + a + 1           a^2 + 1           a^3 + a       a^2 + a + 1     a^3 + a^2 + a a^3 + a^2 + a + 1     a^3 + a^2 + 1           a^3 + 1                 1]
[                1               a^2             a + 1         a^3 + a^2           a^2 + 1       a^2 + a + 1 a^3 + a^2 + a + 1           a^3 + 1                 a               a^3           a^2 + a       a^3 + a + 1           a^3 + a     a^3 + a^2 + a     a^3 + a^2 + 1                 1]

Conjunto del Cuerpo de Goppa: 


[1,
 a,
 a^2,
 a^3,
 a + 1,
 a^2 + a,
 a^3 + a^2,
 a^3 + a + 1,
 a^2 + 1,
 a^3 + a,
 a^2 + a + 1,
 a^3 + a^2 + a,
 a^3 + a^2 + a + 1,
 a^3 + a^2 + 1,
 a^3 + 1,
 1]

---
## Matriz D

In [14]:
D = diagonal_matrix([1/g(L[i]) for i in range(N)])
print ("Matriz D: ")
show(D)

Matriz D: 


[            1             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0]
[            0       a^2 + 1             0             0             0             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             a             0             0             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             0 a^3 + a^2 + a             0             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             0             0           a^2             0             0             0             0             0             0             0             0             0             0             0]
[            0             0             0             0             0   a^2 + a + 1             0             0             0             0             0             0             0             0             0             0]
[            0             0             0             0             0             0   a^3 + a + 1             0             0             0             0             0             0             0             0             0]
[            0             0             0             0             0             0             0   a^2 + a + 1             0             0             0             0             0             0             0             0]
[            0             0             0             0             0             0             0             0         a + 1             0             0             0             0             0             0             0]
[            0             0             0             0             0             0             0             0             0 a^3 + a^2 + 1             0             0             0             0             0             0]
[            0             0             0             0             0             0             0             0             0             0       a^2 + a             0             0             0             0             0]
[            0             0             0             0             0             0             0             0             0             0             0       a^2 + a             0             0             0             0]
[            0             0             0             0             0             0             0             0             0             0             0             0       a^3 + 1             0             0             0]
[            0             0             0             0             0             0             0             0             0             0             0             0             0   a^2 + a + 1             0             0]
[            0             0             0             0             0             0             0             0             0             0             0             0             0             0       a^2 + a             0]
[            0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             1]

---
## Matriz H

In [15]:
H = T*V*D
print ("Matriz H: ")
show(H)

Matriz H: 


[                1           a^2 + 1                 a     a^3 + a^2 + a               a^2       a^2 + a + 1       a^3 + a + 1       a^2 + a + 1             a + 1     a^3 + a^2 + 1           a^2 + a           a^2 + a           a^3 + 1       a^2 + a + 1           a^2 + a                 1]
[                1           a^3 + a               a^3           a^3 + 1         a^3 + a^2                 1     a^3 + a^2 + 1               a^2 a^3 + a^2 + a + 1       a^3 + a + 1                 1                 a     a^3 + a^2 + a           a^2 + 1             a + 1                 1]
[                0                 a               a^2           a^3 + a             a + 1                 1               a^3     a^3 + a^2 + 1           a^2 + 1 a^3 + a^2 + a + 1                 1           a^3 + 1         a^3 + a^2       a^3 + a + 1     a^3 + a^2 + a                 0]

---
## Matriz H Goppa K

In [16]:
H_Goppa_K = matrix(K_, m*H.nrows(),H.ncols())
print ("Matriz H_Goppa_K inicial: ")
show(H_Goppa_K)

for i in range(H.nrows()):
    for j in range(H.ncols()):
        be = bin(eval(H[i,j]._int_repr()))[2:];
        be = '0'*(m-len(be))+be; be = list(be);
        H_Goppa_K[m*i:m*(i+1),j]=vector(map(int,be));

print ("Matriz H_Goppa_K: ")
show(H_Goppa_K)

Matriz H_Goppa_K inicial: 


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Matriz H_Goppa_K: 


[0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0]
[0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0]
[0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0]
[1 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1]
[0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0]
[0 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0]
[0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0]
[1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 0]
[0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0]
[0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0]
[0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0]

In [17]:
show(H_Goppa_K)

[0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0]
[0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 0]
[0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0]
[1 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1]
[0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0]
[0 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0]
[0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0]
[1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 0]
[0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0]
[0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0]
[0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0]

---
## Matriz kernel

In [18]:
Krnl = H_Goppa_K.right_kernel();
print ("Matriz Krnl: ")
show(Krnl)

Matriz Krnl: 


Vector space of degree 16 and dimension 4 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0]
[0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1]

---
## Matriz G

In [19]:
G = Krnl.basis_matrix();
print ("Matriz G: ")
show(G)

Matriz G: 


[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0]
[0 0 1 0 0 0 1 0 1 1 1 1 0 0 1 1]
[0 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1]

---
## Matriz S

In [20]:
S = random_matrix(GF(2), N-m*rango)
while (S.determinant()==0):
    S = random_matrix(GF(2), N-m*rango)

print ("Matriz S: ")
show(S)

Matriz S: 


[0 0 0 1]
[1 0 1 1]
[1 0 0 0]
[1 1 0 1]

---
## Matriz P

In [21]:
rng = range(N)

P = matrix(GF(2),N);
print ("Matriz P inicial: ")
show(P)

for i in range(N):
    p = floor(len(rng)*random());
    P[i,rng[p]]=1;
    rng=[*rng[:p], *rng[p+1:]];

print ("Matriz P: ")
show(P)

Matriz P inicial: 


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Matriz P: 


[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]

---
## Matriz G'

In [22]:
G_prima = S*G*P

print ("Matriz G_prima: ")
show(G_prima)

Matriz G_prima: 


[1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 0]
[0 1 0 0 1 1 0 0 0 0 1 1 1 1 1 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]
[0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0]

In [23]:
print("  Matriz P")
show(P)

print("  Matriz S")
show(S)

print("  Matriz G'")
show(G_prima)

  Matriz P


[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]

  Matriz S


[0 0 0 1]
[1 0 1 1]
[1 0 0 0]
[1 1 0 1]

  Matriz G'


[1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 0]
[0 1 0 0 1 1 0 0 0 0 1 1 1 1 1 1]
[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]
[0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0]

### Cifrado
---

Se genera el mensaje

In [24]:
u = vector(K_,[randint(0,1) for _ in range(G_prima.nrows())])
print("Vector u");
show(u)

Vector u


(1, 1, 1, 0)

cifrado: mG'

In [25]:
c = u*G_prima
print("Vector c");
show(c)

Vector c


(1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1)

Se define un vector de errores

In [26]:
e = vector(K_,N)
e[8] = 1
e[9] = 1
print("Vector de errores e");
show(e)

Vector de errores e


(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0)

Se introduce el vector de errores
</br>Se envía Y = mG' + e

In [27]:
y = c + e
print("Vector codificado y"); show(y)

Vector codificado y


(1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1)

### Descifrado
---

Se calcula el inverso de P
</br>Luego cP^{-1}
</br>Por último se usa un algoritmo de decodificación y se calcula m'S

In [28]:
yP = y*(P.inverse())
show(yP)
yd = decodePatterson(yP)
show(yd)
corregido = (G.transpose()\yd)*S.inverse()
show(corregido)

(0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1)

Error encontrado en la posición: 9
Error encontrado en la posición: 14


(0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1)

(1, 1, 1, 0)